In [1]:
import requests
import re
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import WebDriverException
import time
# For this program to work you need to download the chrome driver that is compatible with your chrome
# To check just go to top right hand corner and go to HELP --> About google chrome
# Download via this website https://sites.google.com/chromium.org/driver/
# After download, unzip it and move the chrome driver doc to Program Files (x86) under This PC and your drive

PATH = "C:\Program Files (x86)\chromedriver.exe"

def has_numbers(inputString):
    return bool(re.search(r'\d', inputString))

def price_getter_srx(address,floor,size,flat_type,unit):     
    # First page
    driver = webdriver.Chrome(PATH)
    driver.get("https://www.srx.com.sg/xvalue-pricing")
    driver.find_element(By.ID,"xvalue-search-box").send_keys(address)
    driver.implicitly_wait(5)
    try: 
        element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "tt-suggestion.tt-selectable"))
            )
        element.click()
    except:
        price = "Unknown"
        driver.quit()
        return(price)

    time.sleep(0.5) 
    element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME,"xValueFloorNo"))
        )
    element.send_keys(str(floor))
    element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME,"xValueUnitNo"))
        )
    element.send_keys(str(unit))
    
    try: 
        find_prop_type = \
        driver.find_element(By.NAME,"xValuePropertyType")
        select_prop_type = Select(find_prop_type)
        select_prop_type.select_by_visible_text(flat_type)
        driver.implicitly_wait(5)
    except:
        price = "Unknown"
        driver.quit()
        return(price)
    time.sleep(0.5)    
    element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.NAME,"xValueSize"))
        ) 
    element.click()
    element.clear()
    element.send_keys(str(size))
    time.sleep(0.5)   
    element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "xvalue-btn.form-control"))
        )
    element.send_keys('\n')
    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login-username"))
        )
        driver.find_element(By.ID, "login-username").send_keys("kanjump5@gmail.com")
        driver.find_element(By.ID,"login-password").send_keys("fsw9@v@rpS9i48r")
        driver.find_element(By.CLASS_NAME,"sign-in-btn").click()
    except:
        try: 
            element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.NAME,"xValueSize"))
            )
            element.click()
            element.clear()
            element.send_keys(str(size))
            element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, "xvalue-btn.form-control"))
                )
            element.send_keys('\n')
            element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "login-username"))
            )
            driver.find_element(By.ID, "login-username").send_keys("kanjump5@gmail.com")
            driver.find_element(By.ID,"login-password").send_keys("fsw9@v@rpS9i48r")
            driver.find_element(By.CLASS_NAME,"sign-in-btn").click()
        except:
            price = "Unknown"
            driver.quit()
            return(price)
    

    # Final page
    try:
        element = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.ID, "xvalue-amount"))
        )
        text_not_empty =  WebDriverWait(driver, 10).until(
            EC.text_to_be_present_in_element((By.ID, "xvalue-amount"),"0")
        )
        output = driver.find_element(By.ID,"xvalue-amount")
        price = output.get_attribute("innerText")    
        
    except:
        price = "Unknown"
    
    finally:
        driver.quit()

    return(price)

In [2]:
# #Read in the data here
# data = pd.read_csv("s2_srx1.csv")
# data["SRX_price"] = "Unknown"
# len(data.index)

654

In [2]:
data = pd.read_csv("s2_srx_results.csv")


,Unnamed: 0.1,Unnamed: 0,address,flat_type,floor_area_sqm,ind,storey_index,floor,unit,SRX_price
0,0,1,239 Bukit Batok East Avenue 5,HDB 3 Room,67,104,4,11,10,"$351,000"
1,1,2,215 Bukit Batok Street 21,HDB 3 Room,67,105,1,2,10,"$327,000"
2,2,3,305 Bukit Batok Street 31,HDB 3 Room,68,106,3,8,10,"$345,000"
3,3,4,363 Bukit Batok Street 31,HDB 3 Room,68,107,2,5,10,"$344,000"
4,4,5,312 Bukit Batok Street 32,HDB 3 Room,68,108,4,11,10,"$351,000"


In [3]:
# This is where the program iterat
for i in range(0,200): # Change the index here, where 
    if (data.iloc[i]["SRX_price"] != "Unknown"):
        continue
    addr = data.iloc[i]["address"]
    floor = data.iloc[i]["floor"]
    size = data.iloc[i]["floor_area_sqm"]
    unit = data.iloc[i]["unit"]
    ind = data.iloc[i]["ind"]
    flat_type = data.iloc[i]["flat_type"]
    if (flat_type == "1 ROOM"):
        price = price_getter_srx(addr,floor,size,"HDB 3 Room",unit)
        data.at[i, 'SRX_price'] = price
    elif (flat_type == "2 ROOM"):
        price = price_getter_srx(addr,floor,size,"HDB 3 Room",unit)
        data.at[i, 'SRX_price'] = price
    else:
        price = price_getter_srx(addr,floor,size,flat_type,unit)
        data.at[i, 'SRX_price'] = price

C:\Users\Josia\AppData\Local\Temp\ipykernel_72212\2968314298.py:24: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [30]:
data.to_csv("s2_srx_results.csv")

In [31]:
temp = data.loc[data.SRX_price=="Unknown"]
len(temp.index)

261